# Gym Sanity (Kairos Alloy)

Quick sanity check that `apps/agents/kairos-gym/kairos_gym.py` can run a short bar-based episode.

This notebook **does not save figures**.


In [ ]:
BASE_CONFIG = "platform/ops/configs/quickstart.toml"
MODE = "sweep"  # recommended for windowed episodes
SPLIT_START = None
SPLIT_END = None
STEPS = 10


In [ ]:
from pathlib import Path
import sys
import importlib.machinery
import importlib.util

repo_root = Path.cwd().resolve()
if not (repo_root / "apps").exists():
    repo_root = repo_root.parent

gym_path = repo_root / "apps" / "agents" / "kairos-gym" / "kairos_gym.py"
spec = importlib.util.spec_from_loader(
    "kairos_gym_impl",
    importlib.machinery.SourceFileLoader("kairos_gym_impl", str(gym_path)),
)
assert spec is not None and spec.loader is not None
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module
spec.loader.exec_module(module)
KairosGymEnv = module.KairosGymEnv

env = KairosGymEnv(
    base_config_path=repo_root / BASE_CONFIG,
    mode=MODE,
    split_start=SPLIT_START,
    split_end=SPLIT_END,
    seed=0,
)

obs, info = env.reset()
obs.shape, info

In [ ]:
total = 0.0
for i in range(STEPS):
    # random policy for sanity
    a = env._rng.randint(0, 2)
    obs, r, terminated, truncated, info = env.step(a)
    total += float(r)
    print(i, "a=", a, "r=", r, "eq=", info.get("equity"), "ts=", info.get("timestamp"))
    if terminated or truncated:
        break

env.close()
total